# Saber Detection

In [ ]:
import cv2
import math
import imageio
import numpy as np
import plotly.express as px
from sklearn.cluster import KMeans

def load_image(idx=0):
    # *** first and last channels were swapped when using OPENCV functions ***
    vid = imageio.get_reader("test_video.mp4",  'ffmpeg')
    vid_iter = vid.iter_data()
    for _ in range(idx+1):
        image = next(vid_iter)
    return image

test_frames = (51, 56, 78, 104, 174, 454)
idx = test_frames[-2]
img = load_image(idx)

### Method 1: BGR thresholding and Hough Lines P

In [ ]:
# reload image to ensure reset when running cell
img = load_image(idx)

# convert to HSV for simplicity
blur = cv2.GaussianBlur(img, (3, 3), 0)
kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
blur = cv2.filter2D(blur, ddepth=-1, kernel=kernel)
m1 = blur[:, :, 1] > 200
m2 = blur[:, :, 2] > 200
m3 = blur[:, :, 0] > 240
mask = (m1 + m2 + m3).astype(np.uint8)

# settings for Hough Lines P
rho = 1                         # distance resolution in pixels of the Hough grid
theta = np.pi / 180             # angular resolution in radians of the Hough grid
threshold = 40                  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 10            # minimum number of pixels making up a line
max_line_gap = 10               # maximum gap in pixels between connectable line segments

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
lines = cv2.HoughLinesP(mask, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)

gray = np.zeros(img.shape[:2], np.uint8)
if isinstance(lines, np.ndarray):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(gray, (x1, y1), (x2, y2), 255, 2)
            length = abs(x1 - x2) + abs(y1 - y2)
            if 100 > length > 40:
                centroid = (int((x1 + x2) / 2), int((y1 + y2) / 2))
                cv2.drawMarker(img, centroid, (0, 255, 0),
                               markerType=cv2.MARKER_CROSS, thickness=2)

# contours, _ = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# for i, contour in enumerate(contours):
#     if 800 > cv2.contourArea(contour) > 60:
#         cv2.drawContours(img, contours, i, 255, -1)
        
px.imshow(img)

### Method 2: Combine BGR and HSV masks

In [ ]:
# reload image to ensure reset when running cell
img = load_image(idx)

b = (img[:, :, 0] > 200).astype(int)
r = (img[:, :, 2] > 180).astype(int)

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

v = (hsv[:, :, 2] > 200).astype(int)
s = cv2.inRange(hsv[:, :, 1],  140, 175)

m1 = np.logical_and(b, s)
m2 = np.logical_and(r, v)
mask = (m1 + m2).astype(np.uint8)

rho = 1                         # distance resolution in pixels of the Hough grid
theta = np.pi / 180             # angular resolution in radians of the Hough grid
threshold = 40                  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 25            # minimum number of pixels making up a line
max_line_gap = 15               # maximum gap in pixels between connectable line segments

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
lines = cv2.HoughLinesP(mask, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)

if isinstance(lines, np.ndarray):
    for line in lines:
        # for x1, y1, x2, y2 in line:
        x1, y1, x2, y2 = line.ravel()
        x_diff = x1 - x2
        y_diff = y1 - y2
        length = (x_diff * x_diff + y_diff * y_diff) ** 0.5
        centroid = (int((x1 + x2) / 2), int((y1 + y2) / 2))
        if 100 > length > 70:
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), thickness=2)
            # cv2.drawMarker(img, centroid, (0, 255, 0),
            #                markerType=cv2.MARKER_CROSS, thickness=2)
            # cv2.putText(img, f"{length:.0f}", centroid, cv2.FONT_HERSHEY_SIMPLEX,
            #             color=(0, 0, 255), fontScale=1, thickness=2)
        
px.imshow(img)

### Method 2.5: Combine BGR and HSV masks with regular Hough Lines

In [ ]:
# reload image to ensure reset when running cell
img = load_image(idx)

b = cv2.inRange(img[:, :, 0], 200, 255)
r = cv2.inRange(img[:, :, 2], 220, 255)

# convert to HSV for more masking options
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
v = cv2.inRange(hsv[:, :, 2], 210, 255)
s = cv2.inRange(hsv[:, :, 1], 140, 175)

# combine masks into one
m1 = cv2.bitwise_and(b, s)
m2 = cv2.bitwise_and(r, v)
mask = cv2.bitwise_or(m1, m2)

# Hough Lines settings
rho = 1                         # distance resolution in pixels of the Hough grid
theta = np.pi / 180             # angular resolution in radians of the Hough grid
threshold = 120                  # minimum number of votes (intersections in Hough grid cell)

# Hough Lines
lines = cv2.HoughLines(mask, rho, theta, threshold)

# process "lines" according to https://www.geeksforgeeks.org/line-detection-python-opencv-houghline-method/
if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv2.line(img, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)
                
px.imshow(img)

In [ ]:
# time non-probabilistic method with optimized params
%timeit cv2.HoughLines(mask, 1, theta, 100)

In [ ]:
# time probabilistic method with optimized params
%timeit cv2.HoughLinesP(mask, 1, theta, 40, np.array([]), 25, 15)

* HoughLines requires processing to get centroids but takes 20% less time to run
* Timescale improvement is not enough to warrant optimization and incorporation (save about 0.2 milliseconds per frame)
* HoughLines all have same length (~2000px, image above) and it's difficult to get the centroids on the sabers

### Method 3: Grayscale clustering and thresholding

K-means clustering on image data is a good way to cluster colors, but speed is an issue so this will never be viable

### Method 4: Reduced input size with Method 2

In [ ]:
# reload image to ensure reset when running cell
img = load_image(test_frames[4])

img = cv2.resize(img, (img.shape[1] // 2, img.shape[0] // 2))
b = (img[:, :, 0] > 200).astype(int)
r = (img[:, :, 2] > 180).astype(int)

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

v = (hsv[:, :, 2] > 170).astype(int)
s = cv2.inRange(hsv[:, :, 1],  140, 175)

m1 = np.logical_and(b, s)
m2 = np.logical_and(r, v)
mask = (m1 + m2).astype(np.uint8)

rho = 1                         # distance resolution in pixels of the Hough grid
theta = np.pi / 180             # angular resolution in radians of the Hough grid
threshold = 20                  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 10            # minimum number of pixels making up a line
max_line_gap = 2               # maximum gap in pixels between connectable line segments

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
lines = cv2.HoughLinesP(mask, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)

if isinstance(lines, np.ndarray):
    for line in lines:
        # for x1, y1, x2, y2 in line:
        x1, y1, x2, y2 = line.ravel()
        x_diff = x1 - x2
        y_diff = y1 - y2
        length = (x_diff * x_diff + y_diff * y_diff) ** 0.5
        centroid = (int((x1 + x2) / 2), int((y1 + y2) / 2))
        if 50 > length > 10:
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), thickness=2)
        
px.imshow(img)